# CWT hyperparameters tuning

Use GPU T4 x 2  
When using GP100, there are XLA errors.  

5 channels (LT, RT, LP, RP, C).

Implementing tf.keras.metrics.KLDivergence().



In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import layers, regularizers
import keras_tuner as kt
import sys
import os


# ----------------------------------------
# Flags for working on my different machines.
# flag_kaggle = True
flag_FW = True
# flag_LN = True

try:
    if flag_kaggle:
        # sys.path.insert(0, '/kaggle/input/hms-lib')
        base_dir = '/kaggle/input/hms-harmful-brain-activity-classification'
        devset_dir = '/kaggle/input/hms-cwt-scalograms-single-numpy-v1'
except:
    pass 

try:
    if flag_FW:
        # sys.path.insert(0, '../lib')
        base_dir = '../../kaggle_data/hms'
        devset_dir = '../data'
except:
    pass 

try:
    if flag_LN:
        # sys.path.insert(0, '../lib')
        base_dir = '../../data/hms'
        devset_dir = '../data'
except:
    pass 
# ----------------------------------------


# path_train = f'{devset_dir}/05_single_cwt_v1_train.npy'
# path_train_items = f'{devset_dir}/05_single_cwt_v1_train_items.npy'
# path_val = f'{devset_dir}/05_single_cwt_v1_val.npy'
# path_val_items = f'{devset_dir}/05_single_cwt_v1_val_items.npy'
# path_test = f'{devset_dir}/05_single_cwt_v1_test.npy'
# path_test_items = f'{devset_dir}/05_single_cwt_v1_test_items.npy'

path_train = f'{devset_dir}/05_reduced_single_cwt_v1_train.npy'
path_train_items = f'{devset_dir}/05_reduced_single_cwt_v1_train_items.npy'
path_val = f'{devset_dir}/05_reduced_single_cwt_v1_val.npy'
path_val_items = f'{devset_dir}/05_reduced_single_cwt_v1_val_items.npy'
path_test = f'{devset_dir}/05_reduced_single_cwt_v1_test.npy'
path_test_items = f'{devset_dir}/05_reduced_single_cwt_v1_test_items.npy'


2024-03-11 10:42:51.416753: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'keras_tuner'

## Data generators

In [3]:
#
# Data generator for training.
#
# coefficients of cwt's arrays
# 5 channels (LP, RP, LT, RP, C)
#

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path_to_items, path_to_data, batch_size=32, n_classes=6, shuffle=True):
        ''' Initialization
        item: [eeg_id, eeg_sub_id, idx in sgrams (1st index), target,
        seizure_vote, lpd_vote, gpd_vote, lrda_vote,
        grda_vote, other_vote]
        '''
        self.n_channels = 5
        # self.n_freqs = 40

        self.data = np.load(path_to_data)
        self.items = np.load(path_to_items)
        self.dim = (self.data.shape[1], self.data.shape[2])
        self.batch_size = batch_size
        self.len = self.data.shape[0]
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.len / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def get_dim(self):
        'Dimensions for the input layer.'
        return (self.dim[0], self.dim[1], self.n_channels)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        true_size = len(indexes)
        X = np.empty((true_size, *self.dim, self.n_channels))
        y = np.empty((true_size, self.n_classes), dtype=float)

        # Generate data
        for i, idx in enumerate(indexes):
            item = self.items[idx]
            # print(item)  # Uncomment for testing.
            X[i,:,:,:] = self.data[np.int32(item[2]), :, :, :]
            # Store solution
            y[i,:] = item[-6:]

        return X, y


# #
# # Test Data generator for predicting
# # 

# class TestDataGenerator(keras.utils.Sequence):
#     'Generates data for Keras'
#     def __init__(self, path_to_items, path_to_data, batch_size=32, n_classes=6, shuffle=False):
#         ''' Initialization
#         item: [eeg_id, eeg_sub_id, idx in sgrams (1st index), target,
#         seizure_vote, lpd_vote, gpd_vote, lrda_vote,
#         grda_vote, other_vote]
#         '''
#         self.n_channels = 5
#         self.data = np.load(path_to_data)
#         self.items = np.load(path_to_items)
#         self.dim = (self.data.shape[1], self.data.shape[2])
#         self.batch_size = batch_size
#         self.len = self.data.shape[0]
#         self.n_classes = n_classes
#         self.shuffle = shuffle
#         self.on_epoch_end()

#     def __len__(self):
#         'Denotes the number of batches per epoch'
#         return int(np.ceil(self.len / self.batch_size))

#     def __getitem__(self, index):
#         'Generate one batch of data'
#         # Generate indexes of the batch
#         indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

#         # Generate data
#         X = self.__data_generation(indexes)

#         return X

#     def get_dim(self):
#         'Dimensions for the input layer.'
#         return (self.dim[0], self.dim[1], self.n_channels)

#     def on_epoch_end(self):
#         'Updates indexes after each epoch'
#         self.indexes = np.arange(self.len)
#         # pass 
        
#     def __data_generation(self, indexes):
#         'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
#         # Initialization
#         true_size = len(indexes)
#         X = np.empty((true_size, *self.dim, self.n_channels))

#         # Generate data
#         for i, idx in enumerate(indexes):
#             item = self.items[idx]
#             # print(item)  # Uncomment for testing.
#             X[i,:,:,:] = self.data[np.int32(item[2]), :, :, :]

#         return X


## HP tuning

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    # learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


build_model(kt.HyperParameters())


In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    # objective="val_accuracy",
    objective=keras_tuner.Objective("val_mean_absolute_error", direction="min"),
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)


In [ ]:
# tuner.search_space_summary()

In [ ]:
tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))


In [ ]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()


In [ ]:
# tuner.results_summary()

## Retrain the model

In [ ]:
# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])
# Fit with the entire dataset.
x_all = np.concatenate((x_train, x_val))
y_all = np.concatenate((y_train, y_val))
model.fit(x=x_all, y=y_all, epochs=1)


In [4]:

def make_model(input_shape, num_classes):
    input_layer = keras.layers.Input(input_shape)

    #max1 = keras.layers.MaxPooling1D(pool_size=2)(input_layer)
    
    conv1 = keras.layers.Conv2D(filters=32, kernel_size=3, padding="same")(input_layer)
    #conv1 = keras.layers.BatchNormalization()(conv1)
    # conv1 = keras.layers.MaxPooling2D(pool_size=8)(conv1)
    conv1 = keras.layers.ReLU()(conv1)
    
    conv2 = keras.layers.Conv2D(filters=64, kernel_size=5, padding="same")(conv1)
    #conv2 = keras.layers.BatchNormalization()(conv2)
    # conv2 = keras.layers.MaxPooling2D(pool_size=8)(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv2D(filters=256, kernel_size=3, padding="same")(conv2)
    #conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.MaxPooling2D(pool_size=2)(conv3)
    conv3 = keras.layers.ReLU()(conv3)

#     conv4 = keras.layers.Conv2D(filters=512, kernel_size=3, padding="same")(conv3)
#     conv4 = keras.layers.BatchNormalization()(conv4)
#     conv4 = keras.layers.MaxPooling2D(pool_size=4)(conv4)
#     conv4 = keras.layers.ReLU()(conv4)

    fltn  = keras.layers.Flatten()(conv3) 
    
    relu1 = keras.layers.Dense(128)(fltn)
    relu1 = keras.layers.ReLU()(relu1)

#     relu2 = keras.layers.Dense(64)(relu1)
#     relu2 = keras.layers.ReLU(64)(relu2)

#     lin = keras.layers.Dense(2)(relu2)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(relu1)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


In [5]:
# Parameters
params = {
    'batch_size': 32,
    'n_classes': 6,
    'shuffle': True
    }

training_generator = DataGenerator(path_train_items, path_train , **params)
validation_generator = DataGenerator(path_val_items, path_val, **params)

print("Observations in training set:", training_generator.__len__()*params['batch_size'])
print("Observations in validation set:", validation_generator.__len__()*params['batch_size'])


Observations in training set: 12192
Observations in validation set: 2176


In [6]:
checkpoint_filepath = 'checkpoint2.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_kl_divergence',
    mode='min',
    save_best_only=True)


In [7]:
opt = keras.optimizers.SGD(
    learning_rate=0.007,
    momentum=0.01,
)

# opt = keras.optimizers.Adam(
#     learning_rate=0.004,
# )

dim = training_generator.get_dim()

model = make_model(input_shape=dim, num_classes=6)

model.load_weights('/kaggle/input/hms-model-cwt-v1/checkpoint.model.keras')

model.compile(optimizer=opt,
            loss=tf.keras.losses.KLDivergence(),
            metrics=[tf.keras.metrics.KLDivergence()])

model.fit(training_generator, epochs=15,
          validation_data=validation_generator,
          callbacks=[model_checkpoint_callback])

Epoch 1/15


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  1/381 ━━━━━━━━━━━━━━━━━━━━ 3:11:27 30s/step - kl_divergence: 0.0373 - loss: 0.0373

I0000 00:00:1710094887.885214      74 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1710094887.906206      74 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


381/381 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step - kl_divergence: 0.0754 - loss: 0.0754

W0000 00:00:1710094980.422426      74 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


381/381 ━━━━━━━━━━━━━━━━━━━━ 135s 276ms/step - kl_divergence: 0.0754 - loss: 0.0754 - val_kl_divergence: 0.6311 - val_loss: 0.6233
Epoch 2/15
381/381 ━━━━━━━━━━━━━━━━━━━━ 75s 196ms/step - kl_divergence: 0.0502 - loss: 0.0503 - val_kl_divergence: 0.6588 - val_loss: 0.6506
Epoch 3/15
381/381 ━━━━━━━━━━━━━━━━━━━━ 75s 196ms/step - kl_divergence: 0.0418 - loss: 0.0418 - val_kl_divergence: 0.6829 - val_loss: 0.6751
Epoch 4/15
381/381 ━━━━━━━━━━━━━━━━━━━━ 75s 196ms/step - kl_divergence: 0.0443 - loss: 0.0443 - val_kl_divergence: 0.6666 - val_loss: 0.6802
Epoch 5/15
381/381 ━━━━━━━━━━━━━━━━━━━━ 75s 196ms/step - kl_divergence: 0.0446 - loss: 0.0446 - val_kl_divergence: 0.7022 - val_loss: 0.6935
Epoch 6/15
381/381 ━━━━━━━━━━━━━━━━━━━━ 75s 196ms/step - kl_divergence: 0.0462 - loss: 0.0462 - val_kl_divergence: 0.6773 - val_loss: 0.6709
Epoch 7/15
381/381 ━━━━━━━━━━━━━━━━━━━━ 75s 196ms/step - kl_divergence: 0.0430 - loss: 0.0430 - val_kl_divergence: 0.6678 - val_loss: 0.6702
Epoch 8/15
381/381 ━━━━

In [8]:
model.save("model_cwt_031001_057.keras")

## Score

In [9]:
# TARGETS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']

# #
# # Test Data generator: for predicting
# # using own test set.
# # (Not for predicting LB)
# #

# class TestDataGenerator(keras.utils.Sequence):
#     'Generates data for Keras'
#     def __init__(self, path_to_items, path_to_data, batch_size=32, n_classes=6, shuffle=False):
#         ''' Initialization
#         item: [eeg_id, eeg_sub_id, idx in sgrams (1st index), target,
#         seizure_vote, lpd_vote, gpd_vote, lrda_vote,
#         grda_vote, other_vote]
#         '''
#         self.n_channels = 5
#         self.data = np.load(path_to_data)
#         self.items = np.load(path_to_items)
#         self.dim = (self.data.shape[1], self.data.shape[2])
#         self.batch_size = batch_size
#         self.len = self.data.shape[0]
#         self.n_classes = n_classes
#         self.shuffle = shuffle
#         self.on_epoch_end()

#     def __len__(self):
#         'Denotes the number of batches per epoch'
#         return int(np.ceil(self.len / self.batch_size))

#     def __getitem__(self, index):
#         'Generate one batch of data'
#         # Generate indexes of the batch
#         indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

#         # Generate data
#         X = self.__data_generation(indexes)

#         return X

#     def get_dim(self):
#         'Dimensions for the input layer.'
#         return (self.dim[0], self.dim[1], self.n_channels)

#     def on_epoch_end(self):
#         'Updates indexes after each epoch'
#         self.indexes = np.arange(self.len)
#         # pass 
        
#     def __data_generation(self, indexes):
#         'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
#         # Initialization
#         true_size = len(indexes)
#         X = np.empty((true_size, *self.dim, self.n_channels))

#         # Generate data
#         for i, idx in enumerate(indexes):
#             item = self.items[idx]
#             # print(item)  # Uncomment for testing.
#             X[i,:,:,:] = self.data[np.int32(item[2]), :, :, :]

#         return X
    
                
# params = {
#     'batch_size': 32,
#     'n_classes': 6,
#     }

# test_generator = TestDataGenerator(path_test_items, path_test, **params)

# y_pred = model.predict(test_generator)

## Scoring without submission

Using a local test set.

In [10]:
# test_items = np.load(path_test_items)
# # test_items = np.load(f'{devset_dir}/03_single_spectrograms_reduced_v1_test_items.npy')
# df_test_items = pd.DataFrame(test_items)
# df_test_items[0] = df_test_items[0].astype(int)

# sub = pd.DataFrame({'eeg_id':df_test_items[0]})
# sub[TARGETS] = np.round(y_pred,6)
# sub.to_csv('submission.csv',index=False)

# df_test_scoring = df_test_items[[0,4,5,6,7,8,9]]
# df_test_scoring.columns = sub.columns
# # df_test_scoring

# score(df_test_scoring, sub, 'eeg_id')

# Uniform probabilities classificator

In [11]:
# test_npy = np.load(path_test)
# test_items = np.load(path_test_items)

# y_pred = np.ones((test_items.shape[0],6),dtype=float)
# y_pred[:,0:4] = y_pred[:,0:4] * 0.167
# y_pred[:,4:] = y_pred[:,4:] * 0.166

# df_test_items = pd.DataFrame(test_items)
# df_test_items[0] = df_test_items[0].astype(int)

# sub = pd.DataFrame({'eeg_id':df_test_items[0]})
# sub[TARGETS] = np.round(y_pred,6)
# # sub.to_csv('submission.csv',index=False)

# df_test_scoring = df_test_items[[0,4,5,6,7,8,9]]
# df_test_scoring.columns = sub.columns

# score(df_test_scoring, sub, 'eeg_id')


# Submit to LB